## 线性回归的Pytorch实现

In [66]:
import torch
from torch import nn
import numpy as np
torch.manual_seed(1) #生成随机数种子

torch.set_default_tensor_type('torch.FloatTensor')

### 随机生成数据集

In [67]:
num_inputs=3
num_examples=1000
real_w=[-6, 3.2, 10.5]  # w1,w2,w3真实值
real_b=1.7  # b真实值

# 生成随机输入，为1000*3矩阵
features=torch.randn(num_examples,num_inputs,dtype=torch.float32)

# 根据真实值生成标签
labels = real_w[0] * features[:,0] 
for i in range(1,num_inputs):
    labels += real_w[i]*features[:,i]
labels += real_b

# 给标签加上噪声
labels+=torch.tensor(np.random.normal(0,0.01,size=labels.size()),dtype=torch.float32)

### 读取数据

In [68]:
import torch.utils.data as Data

batch_size=10

# 将训练数据的特征和标签打包成组合
dataset=Data.TensorDataset(features,labels)

# 随机按Batch读取（代替手写iter）shuffle=True表示随机顺序读取
data_iter=Data.DataLoader(dataset,batch_size,shuffle=True)

### 定义模型网络

In [69]:
# 方法一，继承nn.Moudle自定义网络
class LinearNet(nn.Module):
    def __init__(self, n_feature):
        super(LinearNet, self).__init__()
        self.linear = nn.Linear(n_feature, 1)
    # forward 定义前向传播
    def forward(self, x):
        y = self.linear(x)
        return y
    
net = LinearNet(num_inputs)
# print(net) 使用print可以打印出网络的结构

In [70]:
# 方法2 利用nn.Sequential容器顺序加入层
net=nn.Sequential(
    nn.Linear(num_inputs,1) # 输入num_inputs个数据，输出1个数据
)

### 初始化模型参数

In [71]:
from torch.nn import init

init.normal_(net[0].weight, mean=0, std=0.01) # w初始化为随机采样均值0，标准差0.01的正态分布
init.constant_(net[0].bias, val=0) # b初始化为0

Parameter containing:
tensor([0.], requires_grad=True)

### 定义损失函数

In [72]:
loss=nn.MSELoss()

### 定义优化算法

In [73]:
import torch.optim as optim

optimizer=optim.SGD(net[0].parameters(),lr=0.03)

### 训练模型

In [74]:
num_epochs=3
for epoch in range(1,num_epochs+1):
    for x,y in data_iter:
        output=net(x)
        l=loss(output, y.view(-1,1))  # y.view(-1,1)改变y的形状，1表示改为1列，-1表示根据其他维度（此处为行）推断本维度（列数）
        optimizer.zero_grad() #梯度清零，等价于net.zero_grad()
        l.backward()
        optimizer.step() #迭代模型参数
    print('epoch %d, loss: %f' % (epoch, l.item()))

# 输出结果
print('\n', real_w,'\n',net[0].weight)
print(real_b,'\n',net[0].bias)


epoch 1, loss: 0.000227
epoch 2, loss: 0.000018
epoch 3, loss: 0.000053

 [-6, 3.2, 10.5] 
 Parameter containing:
tensor([[-5.9998,  3.2000, 10.5004]], requires_grad=True)
1.7 
 Parameter containing:
tensor([1.7000], requires_grad=True)
